In [27]:
import sqlite3
import pandas as pd
import hashlib

""" Les guillemets autour de '{}', n'est pas nécessaire dans la requête SQL mais pour s'aasurer qu'il n'y ait aucune erreur due à des noms de tables ayant des caractères spéciaux ou des espaces, c'est une bonne pratique."""


################################## CONNEXION BD POUR DATAS ET HIST STOCKS  ##################################

class FinanceDatabaseStocks:
#Classe pour gérer les interactions avec la base de données SQLite des actifs financiers.

    # Chemin de la base de données (modifiable à un seul endroit)
    def __init__(self, db_path="data.db"):
        self.db_path = db_path

    
    def get_list_stocks(self):
        #Récupérer la liste des entreprises
        with sqlite3.connect(self.db_path) as conn:
            df = pd.read_sql("SELECT DISTINCT Short_Name_Stocks FROM stocks_infos_par_indice", conn)
        return df["Short_Name_Stocks"].tolist()
    
  
    def get_infos_stocks(self):
        with sqlite3.connect(self.db_path) as conn:
            df = pd.read_sql("SELECT * FROM stocks_infos_par_indice", conn)
        # Supprimer les doublons sur la colonne d'identification de l'entreprise
        df = df.drop_duplicates(subset=["Short_Name_Stocks"])
        return df


    def get_prix_date(self, actif):
        #Récupérer les données de l'actif pour le graphique
        with sqlite3.connect(self.db_path) as conn:
            query = "SELECT Date, Close FROM historique_stocks WHERE Short_Name_Stocks = ? ORDER BY Date"
            df = pd.read_sql(query, conn, params=(actif,))
        if not df.empty:
            df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y")
            df = df.set_index("Date").resample("W").last().reset_index()
        return df



################################## CONNEXION BD POUR DATAS ET HIST INDICES  ##################################

class FinanceDatabaseIndice:
#Classe pour gérer les interactions avec la base de données SQLite des actifs financiers.

    # Chemin de la base de données (modifiable à un seul endroit)
    def __init__(self, db_path="data.db"):
        self.db_path = db_path
        

    def get_list_indices(self):
        #Récupérer la liste des entreprises
        with sqlite3.connect(self.db_path) as conn:
            df = pd.read_sql("SELECT DISTINCT Short_Name_Indice FROM indices_infos", conn)
        return df["Short_Name_Indice"].tolist()
    
  
    def get_infos_indices(self):
        with sqlite3.connect(self.db_path) as conn:
            df = pd.read_sql("SELECT * FROM indices_infos", conn)
        return df


    def get_prix_date(self, selected_indice):
        #Récupérer les données de l'actif pour le graphique
        with sqlite3.connect(self.db_path) as conn:
            query = "SELECT Date, Close FROM historique_indices WHERE Short_Name_Indice = ? ORDER BY Date"
            df = pd.read_sql(query, conn, params=(selected_indice,))
        if not df.empty:
            df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y")
            df = df.set_index("Date").resample("W").last().reset_index()
        return df

    
    def get_composition_indice(self, selected_indice):
        try:
            with sqlite3.connect(self.db_path) as conn:
                query = """
               SELECT 
                	s.*
                FROM stocks_infos_par_indice s
                JOIN indices_infos i ON s.Ticker_Indice_Yf = i.Ticker_Indice_Yf
                WHERE i.Short_Name_Indice =  ?
                ORDER BY s.Short_Name_Stocks
                """
                df = pd.read_sql(query, conn, params=(selected_indice,))
            return df
        except Exception as e:
            print(f"Erreur: {e}")
            return pd.DataFrame()


####################################### CALCUL RENDEMENTS ACTIFS #######################################

def calculate_rendement(df, periods):
    """ Calculer les rendements pour chaque période """
    rendement = {}
    for period_months in periods:
        start_date = df["Date"].max() - pd.DateOffset(months=period_months)
        df_period = df[df["Date"] >= start_date]
        if len(df_period) > 1:  # Si on a plus d'une donnée dans la période
            start_close = df_period.iloc[0]["Close"]
            end_close = df_period.iloc[-1]["Close"]
            rendement[f"{period_months} mois"] = "{:.2f}".format((end_close - start_close) / start_close * 100) # arrondie 2 chif
        else:
            rendement[f"{period_months} mois"] = None
    return rendement



####################################### STYLE DU TABLEAU DE RENDEMENT #######################################

def style_rendement(df, periods):
    """ Appliquer un style de couleur sur les rendements """
    def color_rendement(val):
        color = 'green' if float(val) > 0 else ('red' if float(val) < 0 else 'black')
        return f'color: {color}'  
    return df.style.applymap(color_rendement, subset=[f"{p} mois" for p in periods])







In [28]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.colors as pc
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
#from test_connexion_db_datas import *



def calcul_rendement(duree_invest = 1 , somme_investie = 100000, mois_dca = 6, ticker = "S&P 500"):
    
#=============================== On prépare les variables ===============================  
    # Somme à investir par mois
    somme_par_mois = somme_investie / mois_dca

    # Définir les dates de début et de fin
    date_debut = datetime.now() - relativedelta(months=((duree_invest * 12) + 1)) # +1 pour s'assurer d'avoir un mois entier
    date_fin = datetime.now()

    # Création instance de l'objet
    datas_indices = FinanceDatabaseIndice(db_path="data.db")
    
    # Appel méthodes
    liste_indices = datas_indices.get_list_indices()
    infos_indices = datas_indices.get_infos_indices()
    data_financiere = datas_indices.get_prix_date(ticker)
    
    # Télécharger les données financières pour la période
    data_financiere = data_financiere[(data_financiere['Date'] >= date_debut) & (data_financiere['Date'] <= date_fin)]
 
#=============================== On clean et calcul le rendement par mois ===============================
     # Remplace les cases vides par '0'
    if data_financiere.empty:
        return 0, 0
    # Remplace les cases NaN par '0'
    data_financiere = data_financiere.fillna(0)

    # Rendements mensuels en % (avec colonne 'Close' du df 'data_financiere')
    rendements_mois = data_financiere['Close'].pct_change().dropna()

    # Ajout colonne rendemenbt mensuel au df data_financiere
    data_financiere['Rendement du mois'] = rendements_mois

#=============================== On calcul le DCA et le LumpSum ===============================
    # Calcul du rendement DCA
    rendements_dca = []
    portefeuille_dca = 0
    
    # Investissement mensuel pendant la période DCA
    for i in range(min(mois_dca, len(rendements_mois))):
        portefeuille_dca += somme_par_mois * (1 + rendements_mois.iloc[i])
        rendements_dca.append(round(portefeuille_dca, 2))
   
    # Croissance après la période DCA
    for i in range(mois_dca, len(rendements_mois)):
        portefeuille_dca *= (1 + rendements_mois.iloc[i])
        rendements_dca.append(round(portefeuille_dca, 2))

    # Calcul du rendement LP
    rendements_lumpsum = []
    portefeuille_lumpsum = somme_investie
    for i in range(len(rendements_mois)):
        portefeuille_lumpsum *= (1 + rendements_mois.iloc[i])
        rendements_lumpsum.append(round(portefeuille_lumpsum, 1))
    
    # On aligne la taille avec df_rendement, car pct_change() enlève le premier mois
    data_financiere = data_financiere.iloc[1:]  # on enlève le premier mois (NaN dans pct_change)
    data_financiere['Rendement LS'] = rendements_lumpsum
    data_financiere['Rendement DCA'] = rendements_dca

    return data_financiere

df = calcul_rendement(duree_invest = 1 , somme_investie = 100000, mois_dca = 6, ticker = "^GSPC")


In [31]:
################################### DF POUR GRAPHIQUE BAR ###################################
def calcul_rendements_durations(durees=range(1, 26), mois_dca_list=[3, 6, 12, 24], somme_investie=100000, ticker="S&P 500"):
    
    # Je crée mes listes vides
    annees = []
    lumpsum = []
    
    # Je crée une liste vide pour chaque DCA
    listes_dca = []
    for mois in mois_dca_list:
        listes_dca.append([])
    
    # Pour chaque durée
    for duree in durees:
        # J'ajoute l'année
        annees.append(duree)
        
        # Je calcule chaque DCA
        for i, mois in enumerate(mois_dca_list):
            df = calcul_rendement(duree_invest=duree, somme_investie=somme_investie, mois_dca=mois, ticker=ticker)
            if df.empty:
                listes_dca[i].append(None)
            else:
                listes_dca[i].append(round(df["Rendement DCA"].iloc[-1], 1))
        
        # Je calcule LumpSum (je prends le premier DCA)
        df = calcul_rendement(duree_invest=duree, somme_investie=somme_investie, mois_dca=mois_dca_list[0], ticker=ticker)
        if df.empty:
            lumpsum.append(None)
        else:
            lumpsum.append(round(df["Rendement LS"].iloc[-1], 1))
    
    # Je crée mon dictionnaire pour le DataFrame
    data = {'Année': annees}
    
    # J'ajoute chaque colonne DCA
    for i, mois in enumerate(mois_dca_list):
        data[f'DCA ({mois} mois)'] = listes_dca[i]
    
    # J'ajoute la colonne LumpSum
    data['LumpSum'] = lumpsum
    
    # Je crée mon DataFrame
    df_resultats = pd.DataFrame(data)

    
        
    return df_resultats

calcul_rendements_durations(durees=range(1, 26), mois_dca_list=[3, 6, 12, 24], somme_investie=100000, ticker="S&P 500")


NameError: name 'dca_colonnes' is not defined

In [22]:
def calcul_multiple_rendements(durees = [25, 20, 15, 10, 5], mois_dca_list = [3, 6, 12, 24], somme_investie  = 100000, ticker = "S&P 500"):
    
    # Je crée ma liste vide pour stocker tous mes DataFrames
    tous_les_df = []

    # Pour chaque DCA
    for mois in mois_dca_list:
        # Pour chaque durée de chaque DCA
        for duree in durees:
            # Calcule le rendement
            df = calcul_rendement(duree_invest=duree, somme_investie=somme_investie, mois_dca=mois, ticker=ticker)
            df = df.copy()
            # J'ajoute la colonne durée
            df["Durée"] = f"{duree} ans"
            # J'ajoute la colonne mois DCA
            df["Mois DCA"] = f"{mois} mois"
            # J'ajoute ce DataFrame à ma liste de Dataframe
            tous_les_df.append(df)

    df_resultat = pd.concat(tous_les_df)

    return df_resultat

calcul_multiple_rendements(durees = [25, 20, 15, 10, 5], mois_dca_list = [3, 6, 12, 24], somme_investie  = 100000, ticker = "S&P 500")


,Date,Close,Rendement du mois,Rendement LS,Rendement DCA,Durée,Mois DCA
3778,2000-05-28,1378.0200,-0.020562,97943.8,32647.93,25 ans,3 mois
3779,2000-06-04,1477.2600,0.072016,104997.3,68381.81,25 ans,3 mois
3780,2000-06-11,1456.9500,-0.013748,103553.8,101256.86,25 ans,3 mois
3781,2000-06-18,1464.4600,0.005155,104087.6,101778.80,25 ans,3 mois
3782,2000-06-25,1441.4800,-0.015692,102454.2,100181.71,25 ans,3 mois
...,...,...,...,...,...,...,...
5080,2025-05-11,5659.9102,-0.004706,197643.3,174110.08,5 ans,24 mois
5081,2025-05-18,5958.3799,0.052734,208065.8,183291.60,5 ans,24 mois
5082,2025-05-25,5802.8198,-0.026108,202633.6,178506.26,5 ans,24 mois
5083,2025-06-01,5911.6899,0.018762,206435.4,181855.32,5 ans,24 mois


In [30]:
def graphe_barre(df_resultats):

    # Graphique vide
    fig = go.Figure()

    # Couleurs personnalisées
    couleurs = [
        '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b',
        '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'
    ]

    # Détecter les colonnes DCA dynamiquement
    colonnes_dca = []
    for colonne in df_resultats.columns:
        if colonne.startswith("DCA"):
            colonnes_dca.append(colonne)

    # Ajout des barres DCA
    for i, col in enumerate(dca_colonnes):
        fig.add_trace(go.Bar(
            x=df_resultats["Année"],
            y=df_resultats[col],
            name=col,
            marker_color=couleurs[i % len(couleurs)]
        ))

    # Ajout des barres Lump Sum si présente
    if "LumpSum" in df_resultats.columns:
        fig.add_trace(go.Bar(
            x=df_resultats["Année"],
            y=df_resultats["LumpSum"],
            name="Lump Sum",
            marker_color="#000000"
        ))

    # Je configure mon graphique
    fig.update_layout(
        barmode='group',
        title="Comparaison des rendements DCA vs Lump Sum sur différentes périodes",
        xaxis_title="Durée de l'investissement (années)",
        yaxis_title="Valeur finale (€)",
        legend_title="Méthode d'investissement",
        template="plotly_white",
        height=800,
        width=1200
    )
    
    # Je configure l'axe X
    fig.update_xaxes(
        tickmode='array',
        tickvals=list(df_resultats["Année"]),
        tickangle=45,
        autorange='reversed'
    )
    
    # Je configure la police
    fig.update_layout(
        font=dict(family="Arial", size=14)
    )
    #fig.show()
    return fig

graphe_barre(df_resultats)


NameError: name 'df_resultats' is not defined

In [36]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.colors as pc
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
#from src.models.control_datas.connexion_db_datas import *



def calcul_rendement(duree_invest = 1 , somme_investie = 100000, mois_dca = 6, ticker = "S&P 500"):
    
#=============================== On prépare les variables ===============================  
    # Somme à investir par mois
    somme_par_mois = somme_investie / mois_dca

    # Définir les dates de début et de fin
    date_debut = datetime.now() - relativedelta(months=((duree_invest * 12) + 1)) # +1 pour s'assurer d'avoir un mois entier
    date_fin = datetime.now()

    # Création instance de l'objet
    datas_indices = FinanceDatabaseIndice(db_path="data.db")
    
    # Appel méthodes
    liste_indices = datas_indices.get_list_indices()
    infos_indices = datas_indices.get_infos_indices()
    data_financiere = datas_indices.get_prix_date(ticker)
    
    # Télécharger les données financières pour la période
    data_financiere = data_financiere[(data_financiere['Date'] >= date_debut) & (data_financiere['Date'] <= date_fin)]
 
#=============================== On clean et calcul le rendement par mois ===============================
     # Remplace les cases vides par '0'
    if data_financiere.empty:
        return 0, 0
    # Remplace les cases NaN par '0'
    data_financiere = data_financiere.fillna(0)

    # Rendements mensuels en % (avec colonne 'Close' du df 'data_financiere')
    rendements_mois = data_financiere['Close'].pct_change().dropna()

    # Ajout colonne rendemenbt mensuel au df data_financiere
    data_financiere['Rendement du mois'] = rendements_mois

#=============================== On calcul le DCA et le LumpSum ===============================
    # Calcul du rendement DCA
    rendements_dca = []
    portefeuille_dca = 0
    
    # Investissement mensuel pendant la période DCA
    for i in range(min(mois_dca, len(rendements_mois))):
        portefeuille_dca += somme_par_mois * (1 + rendements_mois.iloc[i])
        rendements_dca.append(round(portefeuille_dca, 2))
   
    # Croissance après la période DCA
    for i in range(mois_dca, len(rendements_mois)):
        portefeuille_dca *= (1 + rendements_mois.iloc[i])
        rendements_dca.append(round(portefeuille_dca, 2))

    # Calcul du rendement LP
    rendements_lumpsum = []
    portefeuille_lumpsum = somme_investie
    for i in range(len(rendements_mois)):
        portefeuille_lumpsum *= (1 + rendements_mois.iloc[i])
        rendements_lumpsum.append(round(portefeuille_lumpsum, 1))
    
    # On aligne la taille avec df_rendement, car pct_change() enlève le premier mois
    data_financiere = data_financiere.iloc[1:]  # on enlève le premier mois (NaN dans pct_change)
    data_financiere['Rendement LS'] = rendements_lumpsum
    data_financiere['Rendement DCA'] = rendements_dca

    return data_financiere

df = calcul_rendement(duree_invest = 1 , somme_investie = 100000, mois_dca = 6, ticker = "^GSPC")


################################### DF POUR GRAPHIQUE BAR ###################################
def calcul_rendements_durations(durees=range(1, 26), mois_dca_list=[3, 6, 12, 24], somme_investie=100000, ticker="S&P 500"):
    
    # Je crée mes listes vides
    annees = []
    lumpsum = []
    
    # Je crée une liste vide pour chaque DCA
    listes_dca = []
    for mois in mois_dca_list:
        listes_dca.append([])
    
    # Pour chaque durée
    for duree in durees:
        # J'ajoute l'année
        annees.append(duree)
        
        # Je calcule chaque DCA
        for i, mois in enumerate(mois_dca_list):
            df = calcul_rendement(duree_invest=duree, somme_investie=somme_investie, mois_dca=mois, ticker=ticker)
            if df.empty:
                listes_dca[i].append(None)
            else:
                listes_dca[i].append(round(df["Rendement DCA"].iloc[-1], 1))
        
        # Je calcule LumpSum (je prends le premier DCA)
        df = calcul_rendement(duree_invest=duree, somme_investie=somme_investie, mois_dca=mois_dca_list[0], ticker=ticker)
        if df.empty:
            lumpsum.append(None)
        else:
            lumpsum.append(round(df["Rendement LS"].iloc[-1], 1))
    
    # Je crée mon dictionnaire pour le DataFrame
    data = {'Année': annees}
    
    # J'ajoute chaque colonne DCA
    for i, mois in enumerate(mois_dca_list):
        data[f'DCA ({mois} mois)'] = listes_dca[i]
    
    # J'ajoute la colonne LumpSum
    data['LumpSum'] = lumpsum
    
    # Je crée mon DataFrame
    resultat = pd.DataFrame(data)
        
    return resultat

################################### DF POUR GRAPHIQUE LIGNE ###################################

def calcul_multiple_rendements(durees = [25, 20, 15, 10, 5], mois_dca_list = [3, 6, 12, 24], somme_investie  = 100000, ticker = "S&P 500"):
    
    # Je crée ma liste vide pour stocker tous mes DataFrames
    tous_les_df = []

    # Pour chaque DCA
    for mois in mois_dca_list:
        # Pour chaque durée de chaque DCA
        for duree in durees:
            # Calcule le rendement
            df = calcul_rendement(duree_invest=duree, somme_investie=somme_investie, mois_dca=mois, ticker=ticker)
            df = df.copy()
            # J'ajoute la colonne durée
            df["Durée"] = f"{duree} ans"
            # J'ajoute la colonne mois DCA
            df["Mois DCA"] = f"{mois} mois"
            # J'ajoute ce DataFrame à ma liste de Dataframe
            tous_les_df.append(df)

    df_resultat = pd.concat(tous_les_df)

    return df_resultat



################################### GRAPH BARRE ###################################

def graphe_barre(df_resultats):

    # Graphique vide
    fig = go.Figure()

    # Couleurs personnalisées
    couleurs = [
        '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b',
        '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'
    ]

    # Détecter les colonnes DCA dynamiquement
    colonnes_dca = []
    for colonne in df_resultats.columns:
        if colonne.startswith("DCA"):
            colonnes_dca.append(colonne)

    # Ajout des barres DCA
    for i, col in enumerate(dca_colonnes):
        fig.add_trace(go.Bar(
            x=df_resultats["Année"],
            y=df_resultats[col],
            name=col,
            marker_color=couleurs[i % len(couleurs)]
        ))

    # Ajout des barres Lump Sum si présente
    if "LumpSum" in df_resultats.columns:
        fig.add_trace(go.Bar(
            x=df_resultats["Année"],
            y=df_resultats["LumpSum"],
            name="Lump Sum",
            marker_color="#000000"
        ))

    # Je configure mon graphique
    fig.update_layout(
        barmode='group',
        title="Comparaison des rendements DCA vs Lump Sum sur différentes périodes",
        xaxis_title="Durée de l'investissement (années)",
        yaxis_title="Valeur finale (€)",
        legend_title="Méthode d'investissement",
        template="plotly_white",
        height=800,
        width=1200
    )
    
    # Je configure l'axe X
    fig.update_xaxes(
        tickmode='array',
        tickvals=list(df_resultats["Année"]),
        tickangle=45,
        autorange='reversed'
    )
    
    # Je configure la police
    fig.update_layout(
        font=dict(family="Arial", size=14)
    )
    #fig.show()
    return fig


################################### GRAPH LINE ###################################

def graphe_line(df, somme_investie=100000):
    import plotly.graph_objects as go
    fig = go.Figure()
    
    couleurs_dca = pc.qualitative.Bold
    couleurs_lump = pc.qualitative.Dark24
    durees = sorted(df['Durée'].unique())

    trace_info = []  # Pour stocker (durée, nom_trace)

    
    # Traces DCA
    for i, (duree, mois) in enumerate(df.groupby(['Durée', 'Mois DCA'])):
        if mois['Mois DCA'].iloc[0] != 1:  # Exclure LumpSum
            nom_trace = f"DCA {mois['Mois DCA'].iloc[0]} - {mois['Durée'].iloc[0]} ans"
            
            fig.add_trace(go.Scatter(
                x=mois['Date'],
                y=mois['Rendement DCA'],
                mode='lines',
                name=nom_trace,
                line=dict(width=1.5, dash='dash', color=couleurs_dca[i % len(couleurs_dca)]),
                hovertemplate="Date: %{x}<br>Valeur: %{y:,.0f}€<extra></extra>"
            ))
            trace_info.append(mois['Durée'].iloc[0])  # Durée

    # Traces LumpSum
    for j, duree in enumerate(df['Durée'].unique()):
        for mois_dca in df['Mois DCA'].unique():
            df_filtered = df[(df['Durée'] == duree) & (df['Mois DCA'] == mois_dca)]
        nom_trace = f"LumpSum - {duree} ans"
        
        fig.add_trace(go.Scatter(
            x=df_filtered['Date'],
            y=df_filtered['Rendement LS'],
            mode='lines',
            name=nom_trace,
            line=dict(width=2, color=couleurs_lump[j % len(couleurs_lump)]),
            hovertemplate="Date: %{x}<br>Valeur: %{y:,.0f}€<extra></extra>"
        ))
        trace_info.append(duree)  # Durée

    
    # Création des boutons avec filtrage par durée (DCA + LumpSum)
    boutons_menu = []

    for duree in durees:
        visible_traces = [d == duree for d in trace_info]
        boutons_menu.append(dict(
            label=f"{duree} ans",
            method="update",
            args=[
                {"visible": visible_traces},
                {"title": f"Performance DCA vs LumpSum - {duree} ans"}
            ]
        ))

    # Bouton "Tout voir"
    boutons_menu.insert(0, dict(
        label="Tout voir",
        method="update",
        args=[
            {"visible": [True] * len(trace_info)},
            {"title": f"Performance DCA vs LumpSum (Investissement: {somme_investie:,.0f}€)"}
        ]
    ))

    # Layout
    fig.update_layout(
        title=dict(
            text=f"Performance DCA vs LumpSum (Investissement: {somme_investie:,.0f}€)",
            x=0.5,
            y=0.95,  # Monte le titre
            xanchor='center',
            yanchor="top",
            font=dict(size=20)
        ),
        xaxis=dict(
            title="Date",
            tickangle=-45,
            tickformat="%Y",
            dtick="M12",  # Un tick tous les 12 mois
            showgrid=True,
            gridcolor="LightGrey"
        ),
        yaxis_title="Valeur du portefeuille (€)",
        legend_title="Stratégie",
        template="plotly_white",
        hovermode="x unified",
        height=700,
        width=1200,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.5,
            xanchor="center",
            x=0.5
        ),
        updatemenus=[
            dict(
                type="buttons",  # Boutons côte à côte
                direction="right",
                buttons=boutons_menu,
                showactive=True,
                x=0.5,
                xanchor="center",
                y=1.1,
                yanchor="top"
            )
        ]
    )
    
    #fig.show()
    return fig


# 1. Créer le DataFrame des rendements
df_resultats = calcul_rendements_durations(
    durees=range(1, 26),
    mois_dca_list=[3, 6, 12, 24],
    somme_investie=100000,
    ticker="^GSPC"
)

# 2. Vérifier son contenu
print(df_resultats.head())

# 3. Générer et afficher le graphique
fig = graphe_barre(df_resultats)
fig.show()




AttributeError: 'tuple' object has no attribute 'empty'